In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import xarray as xr


In [2]:
#Replace with file path for this directory on your machine
!pip install -e /Users/justinmaynard/Documents/GitHub/assetraMP/ 


Obtaining file:///Users/justinmaynard/Documents/GitHub/assetraMP
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for assetra (pyproject.toml) ... done
  Created wheel for assetra: filename=assetra-1.0.3-py3-none-any.whl size=2689 sha256=dbec1acd386afa31ce06d71424f177f8781dc8432d27f26b9e6f1bc595d1c07a
  Stored in directory: /private/var/folders/zd/3zqsbnbn5lg29wjyl0df33l80000gn/T/pip-ephem-wheel-cache-jp6hfbml/wheels/c0/f5/c9/3b09bf7f2c6beadc5d6cde6ca7e50094804a4b90493c211e80
Successfully built assetra
  Attempting uninstall: assetra
    Found existing installation: assetra 1.0.3
    Uninstalling assetra-1.0.3:
      Successfully uninstalled assetra-1.0.3


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import xarray as xr
import os 
import shutil


p = Path(".")
root_folder = p.cwd().parent 
git_data_folder = root_folder/   'git_data'
gen_data_folder = git_data_folder / 'gen_forecasts' 
scripts_foler = root_folder/ 'scripts'
outside_data_folder = root_folder / 'outside_data'
gen_data_folder = git_data_folder / 'gen_forecasts'
historical_load_data = git_data_folder / 'historical_load'
load_forecast_data = git_data_folder / 'load_forecasts'

scratch_folder = root_folder  / 'scratch'

#### Demand Data

In [4]:
#Function to load demand data
def load_pjm_cleaned_hourly_demand(
        pjm_demand_file: Path,
        start_hour: datetime,
        end_hour: datetime) -> xr.DataArray:
    """Return hourly demand data as formatted data array.
    To use this function, download cleaned demand data from:

    https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data

    Args:
        eia_930_cleaned_demand_file (Path): Path to hourly demand file
        start_hour (datetime): First timestamp to include
        end_hour (datetime): Last timestamp to include (inclusive)

    Returns:
        xr.DataArray: Hourly demand array with time dimension and datetime coordinates.
    """
    # read demand file
    pjm_demand_df = pd.read_csv(
        pjm_demand_file,
        usecols=["datetime_beginning_ept", "mw"],
        index_col="datetime_beginning_ept",
        parse_dates=True,
    )

    # remove leap day
    #pjm_demand_df = pjm_demand_df[~((pjm_demand_df.index.month == 2) & (pjm_demand_df.index.day == 29))]


    # keep cleaned demand demand
    pjm_hourly_demand_pd = pjm_demand_df["mw"].loc[start_hour:end_hour]

    # convert to xr.DataArray
    pjm_hourly_demand = xr.DataArray(
        data=pjm_hourly_demand_pd.values,
        coords=dict(
            time=pjm_hourly_demand_pd.index.values
        )
    )
    return pjm_hourly_demand

In [5]:
#Load demand data
pjm_cleaned_demand_file = Path( historical_load_data /  "pjm_load_2020.csv")
hourly_demand = load_pjm_cleaned_hourly_demand(
	pjm_cleaned_demand_file,
	start_hour="2020-01-01 00:00:00",
	end_hour="2020-12-31 23:00:00"
)

In [6]:
from assetra.system import EnergySystem
from assetra.system import EnergySystemBuilder

builder = EnergySystemBuilder()
unit_count = 0


In [7]:
# create demand unit
from assetra.units import DemandUnit

builder.add_unit(
    DemandUnit(
        id=unit_count,
        hourly_demand=hourly_demand
    )
)
unit_count += 1

#### MERRA Data

In [8]:
pow_gen_file_2020 = Path(outside_data_folder / "power_gen_data" / "pjm_power_generation_2020.nc")
pow_gen_dataset_2020 = xr.open_dataset(pow_gen_file_2020)


In [9]:

pow_gen_dataset = pow_gen_dataset_2020

def get_nearest_hourly_profile(
    latitude: float,
    longitude: float,
    array: xr.DataArray
) -> xr.DataArray:
    """Return time series corresponding to the nearest coordinate in a
    MERRA power generation data array.

    Args:
        latitude (float): Latitude relative to equator in degrees
        start_hour (datetime): Longitude relative to meridian in degrees
        array (xr.DataArray): "solar_capacity_factor", "wind_capacity_factor",
            or "temperature"

    Returns:
        xr.DataArray: Array with time dimension and datetime coordinates.
    """
    return array.sel(
            lat=latitude, 
            lon=longitude, 
            method="nearest"
        ).squeeze(drop=True)

def get_merra_power_generation_solar_cf(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["solar_capacity_factor"])

def get_merra_power_generation_wind_cf(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["wind_capacity_factor"])

def get_merra_power_generation_temperature(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["temperature"])

In [10]:
import pandas as pd

# load temperature dependent outage rate (tdfor) table
tdfor_table_file = Path( git_data_folder / "temperature_dependent_outage_rates_copy.csv")
tdfor_table = pd.read_csv(tdfor_table_file, index_col=0)
tdfor_table = tdfor_table / 100 # percentages stored as integers

# create mapping table for tdfor table
tech_categories = {
    "CC" : ["Natural Gas Fired Combined Cycle"],
    "CT" : ["Natural Gas Fired Combustion Turbine","Landfill Gas"],
    "DS" : ["Natural Gas Internal Combustion Engine"],
    "ST" : ["Conventional Steam Coal","Natural Gas Steam Turbine"],
    "NU" : ["Nuclear"],
    "HD" : ["Conventional Hydroelectric","Solar Thermal without Energy Storage",
                   "Hydroelectric Pumped Storage","Solar Thermal with Energy Storage","Wood/Wood Waste Biomass"],
    "PR": ["Perfect Resource"]
}

# create mapping from technology to category
tech_mapping = {tech : cat for cat, techs in tech_categories.items() for tech in techs}

def get_hourly_forced_outage_rate(hourly_temperature: xr.DataArray, technology: str) -> xr.DataArray:
    # index tdfor table by tech
    tdfor_map = tdfor_table[tech_mapping.get(technology, "Other")]
    map_temp_to_for = lambda hourly_temperature: tdfor_map.iloc[
            tdfor_map.index.get_indexer(hourly_temperature, method="nearest")
        ]
    return xr.apply_ufunc(
        map_temp_to_for,
        hourly_temperature
    ).rename("hourly_forced_outage_rate")


### Original EUE: 2025 supply, 2020 load, 2020 MERRA

#### Supply Data

In [11]:
#Test for just one year
thermal_generators = pd.read_csv(gen_data_folder / "thermal_2025.csv") #, index_col=0)
wind_generators = pd.read_csv(gen_data_folder / "wind_2025.csv")#, index_col=0)
solar_generators = pd.read_csv(gen_data_folder / "solar_2025.csv")#, index_col=0)
storage_generators = pd.read_csv(gen_data_folder / "batteries_2025.csv")#, index_col=0)
thermal_generators.reset_index(inplace=True)
storage_generators.reset_index(inplace=True)
solar_generators.reset_index(inplace=True)
wind_generators.reset_index(inplace=True)
thermal_generators.loc[(thermal_generators['MC'].isnull()), "MC"] = 9999


#### Build Units

In [12]:
from assetra.units import StochasticUnit


for _, generator in thermal_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["LAT"],
        generator["LON"]
    )

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # get hourly capacity
    hourly_capacity = ( 
        xr.ones_like(hourly_temperature).rename("hourly_capacity") 
        * generator["Nameplate Capacity (MW)"]
    )

    # create assetra energy unit
    thermal_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"],
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(thermal_unit)

In [13]:

# add solar 
for _, generator in solar_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["LAT"],
        generator["LON"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_solar_cf(
        generator["LAT"],
        generator["LON"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    solar_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"], #0,
            )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(solar_unit)

# add wind
for _, generator in wind_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        
        generator["LAT"],
        generator["LON"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_wind_cf(
        generator["LAT"],
        generator["LON"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])


    # create assetra energy unit
    wind_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"],# 0,
            )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(wind_unit)

In [14]:
from assetra.units import StorageUnit

STORAGE_EFFICIENCY = 0.85

for _, generator in storage_generators.iterrows():
    storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"]
    if storage_duration < 4:
        storage_class = '4'
    elif 4 <= storage_duration < 6:
        storage_class = '4'
    elif 6 <= storage_duration < 8:
        storage_class = '8'
    elif 8 <= storage_duration < 10:
        storage_class = '8'
    else:
        storage_class = '4'

    storage_unit = StorageUnit(
        id=unit_count,
        nameplate_capacity=generator["Nameplate Capacity (MW)"],
        charge_rate=generator["Nameplate Capacity (MW)"],
        discharge_rate=generator["Nameplate Capacity (MW)"],
        charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
        roundtrip_efficiency = STORAGE_EFFICIENCY,
        storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"],
        storage_class = storage_class       
    )
    unit_count += 1

        # add unit to energy system
    builder.add_unit(storage_unit)
    #class_to_test_df = pd.DataFrame(class_to_test_lst, columns=storage_generators.columns)

In [15]:
from assetra.system import EnergySystem
from assetra.system import EnergySystemBuilder

In [16]:
system_dir = Path(scratch_folder / "pjm_energy_system_TEST")

if system_dir.exists():
    print("removing existing system directory")
    shutil.rmtree(system_dir)

energy_system = builder.build()
energy_system.save(system_dir)


removing existing system directory


In [18]:
energy_system.size

4039

In [17]:
#energy_system = EnergySystem()
#energy_system.load(system_dir)

In [19]:
print("# of Units:", energy_system.size)
system_cap = energy_system.system_capacity
print("Sys. Capacity (MW):", round(system_cap))

# of Units: 4039
Sys. Capacity (MW): 216459


In [20]:
from assetra.simulation import ProbabilisticSimulation

simulation = ProbabilisticSimulation(
    start_hour="2020-01-01 00:00:00",
    end_hour="2020-12-31 23:00:00",
    trial_size=1
)

simulation.assign_energy_system(energy_system)
simulation.run()

In [21]:
from assetra.metrics import ExpectedUnservedEnergy

# instantiate eue model
eue_model = ExpectedUnservedEnergy(simulation)
eue = eue_model.evaluate()

print("System EUE:", round(eue, 2), "MWh")

System EUE: 104902817.32 MWh


### Additional supply EUE: 2026 battery supply, 2020 load, 2020 MERRA

#### Supply Data

In [22]:
#Higher battery storage with additional capacity to test
storage_generators_2026 = pd.read_csv(gen_data_folder / "batteries_2026.csv")#, index_col=0)

#Calculating additional batteries in 2026
batteries_2026_additional = storage_generators_2026.merge(storage_generators, how='outer', indicator=True)
batteries_2026_additional = batteries_2026_additional[batteries_2026_additional['_merge'] == 'left_only'].drop(columns=['_merge'])


In [50]:
batteries_2026_additional.drop(columns=['index'])

,Technology,ISORTO,Plant Code,Nameplate Capacity (MW),Nameplate Energy Capacity (MWh),LAT,LON,PLHTIAN,PLNGENAN,FC,MC
59,Short-duration batteries,PJM,80406,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0
60,Short-duration batteries,PJM,80407,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0
61,Short-duration batteries,PJM,80408,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0
62,Short-duration batteries,PJM,80409,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0
63,Short-duration batteries,PJM,80410,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0
64,Short-duration batteries,PJM,80411,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0
65,Short-duration batteries,PJM,80412,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0
66,Short-duration batteries,PJM,80413,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0
67,Short-duration batteries,PJM,80414,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0
68,Short-duration batteries,PJM,80415,11.487097,45.948387,41.7678,-88.8867,0.0,,0.0,0.0


#### Build Units

In [23]:
from assetra.units import StorageUnit

STORAGE_EFFICIENCY = 0.85

for _, generator in batteries_2026_additional.iterrows():
    storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"]
    if storage_duration < 4:
        storage_class = '4'
    elif 4 <= storage_duration < 6:
        storage_class = '4'
    elif 6 <= storage_duration < 8:
        storage_class = '8'
    elif 8 <= storage_duration < 10:
        storage_class = '8'
    else:
        storage_class = '4'

    storage_unit = StorageUnit(
        id=unit_count,
        nameplate_capacity=generator["Nameplate Capacity (MW)"],
        charge_rate=generator["Nameplate Capacity (MW)"],
        discharge_rate=generator["Nameplate Capacity (MW)"],
        charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
        roundtrip_efficiency = STORAGE_EFFICIENCY,
        storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"],
        storage_class = storage_class       
    )
    unit_count += 1

        # add unit to energy system
    builder.add_unit(storage_unit)
    #class_to_test_df = pd.DataFrame(class_to_test_lst, columns=storage_generators.columns)

In [24]:
energy_system.size

4039

In [27]:
#system_dir = Path(scratch_folder / "pjm_energy_system_TEST_Additional")

#energy_system = builder.build()
#energy_system.save(system_dir)


In [34]:
#energy_system = EnergySystem()
#energy_system.load(system_dir)

In [26]:
print("# of Units:", energy_system.size)
system_cap_additional =energy_system.system_capacity
print("Sys. Capacity (MW):", round(system_cap_additional))

# of Units: 4095
Sys. Capacity (MW): 217102


In [28]:
from assetra.simulation import ProbabilisticSimulation

simulation = ProbabilisticSimulation(
    start_hour="2020-01-01 00:00:00",
    end_hour="2020-12-31 23:00:00",
    trial_size=1
)

simulation.assign_energy_system(energy_system)
simulation.run()

In [29]:
from assetra.metrics import ExpectedUnservedEnergy

# instantiate eue model
eue_model = ExpectedUnservedEnergy(simulation)
eue_additional = eue_model.evaluate()

print("System EUE:", round(eue_additional, 2), "MWh")

System EUE: 104014068.93 MWh


In [30]:
print("Reduced EUE:", round(eue - eue_additional,2), "MWh from", round((system_cap_additional - system_cap),2), "additional battery MW" )

Reduced EUE: 888748.39 MWh from 643.28 additional battery MW


### Scenario 3 - "Perfect additional 2026 supply", 2020 load, 2020 MERRA

#### Supply Data

In [31]:
#Creating perfect supply equal to the MW added above

cap_to_create = 643.28 #MW
perfect_cap = pd.DataFrame({
    'Technology':['Perfect Resource'],
    'ISORTO':['PJM'],
    'Plant Code': 123456789,
    'Nameplate Capacity (MW)': cap_to_create,
    'LAT':[39.5], #dummy
    'LON':[-86.875], #dummy
    'PLHTIAN':[0],
    'FC':[0],
    'MC':[0]
})
perfect_cap.reset_index()
perfect_cap


,Technology,ISORTO,Plant Code,Nameplate Capacity (MW),LAT,LON,PLHTIAN,FC,MC
0,Perfect Resource,PJM,123456789,643.28,39.5,-86.875,0,0,0


#Build Units

In [32]:

builder = EnergySystemBuilder()
unit_count = 0

# create demand unit
from assetra.units import DemandUnit

builder.add_unit(
    DemandUnit(
        id=unit_count,
        hourly_demand=hourly_demand
    )
)
unit_count += 1

In [33]:
#Test for just one year
thermal_generators = pd.read_csv(gen_data_folder / "thermal_2025.csv") #, index_col=0)
wind_generators = pd.read_csv(gen_data_folder / "wind_2025.csv")#, index_col=0)
solar_generators = pd.read_csv(gen_data_folder / "solar_2025.csv")#, index_col=0)
storage_generators = pd.read_csv(gen_data_folder / "batteries_2025.csv")#, index_col=0)
thermal_generators.reset_index(inplace=True)
storage_generators.reset_index(inplace=True)
solar_generators.reset_index(inplace=True)
wind_generators.reset_index(inplace=True)
thermal_generators.loc[(thermal_generators['MC'].isnull()), "MC"] = 9999


In [34]:
from assetra.units import StochasticUnit


for _, generator in thermal_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["LAT"],
        generator["LON"]
    )

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # get hourly capacity
    hourly_capacity = ( 
        xr.ones_like(hourly_temperature).rename("hourly_capacity") 
        * generator["Nameplate Capacity (MW)"]
    )

    # create assetra energy unit
    thermal_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"],
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(thermal_unit)

In [35]:

# add solar 
for _, generator in solar_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["LAT"],
        generator["LON"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_solar_cf(
        generator["LAT"],
        generator["LON"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    solar_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"], #0,
            )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(solar_unit)

# add wind
for _, generator in wind_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        
        generator["LAT"],
        generator["LON"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_wind_cf(
        generator["LAT"],
        generator["LON"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])


    # create assetra energy unit
    wind_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"],# 0,
            )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(wind_unit)

In [36]:
#Changing battery storage back to original (2025)

from assetra.units import StorageUnit

STORAGE_EFFICIENCY = 0.85

for _, generator in storage_generators.iterrows():
    storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"]
    if storage_duration < 4:
        storage_class = '4'
    elif 4 <= storage_duration < 6:
        storage_class = '4'
    elif 6 <= storage_duration < 8:
        storage_class = '8'
    elif 8 <= storage_duration < 10:
        storage_class = '8'
    else:
        storage_class = '4'

    storage_unit = StorageUnit(
        id=unit_count,
        nameplate_capacity=generator["Nameplate Capacity (MW)"],
        charge_rate=generator["Nameplate Capacity (MW)"],
        discharge_rate=generator["Nameplate Capacity (MW)"],
        charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
        roundtrip_efficiency = STORAGE_EFFICIENCY,
        storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"],
        storage_class = storage_class       
    )
    unit_count += 1

        # add unit to energy system
    builder.add_unit(storage_unit)
    #class_to_test_df = pd.DataFrame(class_to_test_lst, columns=storage_generators.columns)

In [37]:
from assetra.units import StochasticUnit

for _, generator in perfect_cap.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["LAT"],
        generator["LON"]
    )

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # get hourly capacity
    hourly_capacity = ( 
       xr.ones_like(hourly_temperature).rename("hourly_capacity") 
      * generator["Nameplate Capacity (MW)"]
    )

    # create assetra energy unit
    perfect_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"],
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(perfect_unit)

In [38]:
system_dir = Path(scratch_folder / "pjm_energy_system_TEST_perfect")

if system_dir.exists():
    print("removing existing system directory")
    shutil.rmtree(system_dir)

energy_system = builder.build()
energy_system.save(system_dir)


removing existing system directory


In [39]:
energy_system = EnergySystem()
energy_system.load(system_dir)

In [40]:
print("# of Units:", energy_system.size)
system_cap_perfect =energy_system.system_capacity
print("Sys. Capacity (MW):", round(system_cap_perfect))

# of Units: 4040
Sys. Capacity (MW): 217102


In [41]:
from assetra.simulation import ProbabilisticSimulation

simulation = ProbabilisticSimulation(
    start_hour="2020-01-01 00:00:00",
    end_hour="2020-12-31 23:00:00",
    trial_size=1
)

simulation.assign_energy_system(energy_system)
simulation.run()

In [42]:
from assetra.metrics import ExpectedUnservedEnergy

# instantiate eue model
eue_model = ExpectedUnservedEnergy(simulation)
eue_perfect = eue_model.evaluate()

print("System EUE:", round(eue_perfect, 2), "MWh")

System EUE: 102149335.56 MWh


In [43]:
print("Reduced EUE:", round(eue_additional - eue_perfect,2), "MWh from", round((system_cap_perfect - system_cap_additional),2), "additional MW" )

Reduced EUE: 1864733.38 MWh from 0.0 additional MW


In [44]:
print(eue)
print(eue_additional)
print(eue_perfect)

104902817.32125127
104014068.93479744
102149335.55900434


In [52]:
eue_perfect_reuction =  eue_perfect - eue
print((f"EUE Perfect Reduction {round(eue_perfect_reuction, 2)}"))
eue_class_reduction = eue_additional - eue
print((f"EUE Storage Class Reduction {round(eue_class_reduction,2)}"))

eue_class_rating = eue_class_reduction / eue_perfect_reuction
print((f"EUE Class Rating {round(eue_class_rating * 100,2)}"))

EUE Perfect Reduction -2753481.76
EUE Storage Class Reduction -888748.39
EUE Class Rating 32.28


#### ELCC Class Rating

In [46]:
#Not correct formula I belive
EUE_battery_2025 = eue_additional/eue_perfect
EUE_battery_2025

1.0182549731291788